In [1]:
import sys

sys.path.insert(
    1, "..\\utilities\\"
)  # adds utilities folder to path so we can import modules from it, won't be needed after packaging

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import loading_utils as load

participant_list = [200, 201, 202, 204, 205, 206, 207, 209, 210, 211, 212, 213]

In [9]:
data_dir = "./results/new/resampled/"
trial_stats_dict = {}
for participant_id in participant_list:
    data_path = os.path.join(data_dir, str(participant_id) + "_resampled_data.csv")
    data_df_new = pd.read_csv(data_path)

    groupby_df = (
        data_df_new[
            ["Block", "Trial no", "Trial type", "Trial phase", "Stim eye - Size Mm"]
        ]
        .groupby(["Block", "Trial no", "Trial type"])
        .agg(["count", "size"])
        .reset_index()
    )
    groupby_df[("Stim eye - Size Mm", "count/size ratio")] = (
        groupby_df[("Stim eye - Size Mm", "count")]
        / groupby_df[("Stim eye - Size Mm", "size")]
    )
    groupby_df[("Stim eye - Size Mm", "more than 50%")] = (
        groupby_df[("Stim eye - Size Mm", "count/size ratio")] >= 0.5
    )
    trial_stats_dict[str(participant_id)] = (
        groupby_df[
            [
                ("Block", ""),
                ("Trial no", ""),
                ("Trial type", ""),
                ("Stim eye - Size Mm", "count/size ratio"),
                ("Stim eye - Size Mm", "more than 50%"),
            ]
        ]
        .groupby([("Block", ""), ("Trial type", "")])
        .agg(
            {
                ("Stim eye - Size Mm", "count/size ratio"): [
                    "mean",
                    "std",
                    "min",
                    "max",
                ],
                ("Stim eye - Size Mm", "more than 50%"): "sum",
                ("Trial no", ""): "count",
            }
        )
        .reset_index()
    )
    data_df_new = []
    groupby_df = []

## Statistic of available data for each participant per block, per stimulus, for all trial phases combined

count/size ratio - number of samples without nan measurements/ number of samples overall with nan measurements

The above is calculated for each trial, grouped by trial type and trial phase. Then the mean, standard deviation, minimum and maximum are calculated for each group.

more than 50% - number of trials with count/size ratio higher or equal to 0.5

Trial no - count - overall number of trials

### Participant 200

In [12]:
trial_stats_dict["200"]

Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.585586  0.002329  0.580184  0.588175   
1      0        l-m           0.586488  0.001821  0.582227  0.588355   
2      0        lms           0.587072  0.001605  0.583673  0.588957   
3      0        mel           0.585689  0.002094  0.582227  0.588776   
4      0          s           0.586675  0.001264  0.583673  0.588175   
5      1       flux           0.585222  0.003024  0.577551  0.588355   
6      1        l-m           0.586165  0.001322  0.583078  0.587334   
7      1        lms           0.586926  0.001700  0.582653  0.589377   
8      1        mel           0.586932  0.001138  0.584867  0.588355   
9      1          s           0.585569  0.001390  0.582822  0.587156   
10     2       flux           0.584346  0.002401  0.580020  0.587156   
11     2        l-m           0.585527  0.003524  0.576531  0.588776   
12     2        lms           0.585912  0.001373  0.582653  0.587334   
13     2        mel           0.584176  0.004378  0.574055  0.588355   
14     2          s           0.584286  0.005321  0.572449  0.587755   
15     3       flux           0.587287  0.001233  0.584694  0.588776   
16     3        l-m           0.586837  0.001721  0.583673  0.588776   
17     3        lms           0.587577  0.001110  0.585890  0.589377   
18     3        mel           0.587281  0.001088  0.585291  0.589377   
19     3          s           0.586200  0.001617  0.582653  0.587755   
20     4       flux           0.586591  0.001416  0.583673  0.588355   
21     4        l-m           0.585774  0.002568  0.581205  0.588776   
22     4        lms           0.586969  0.001077  0.584694  0.588355   
23     4        mel           0.586668  0.001503  0.584270  0.588355   
24     4          s           0.585588  0.003371  0.576531  0.587755   
25     5       flux           0.583920  0.003722  0.577982  0.588175   
26     5        l-m           0.583036  0.007311  0.563265  0.588355   
27     5        lms           0.584327  0.003009  0.577982  0.587334   
28     5        mel           0.584548  0.003450  0.579162  0.588175   
29     5          s           0.582381  0.004296  0.573904  0.586735   
30     6       flux           0.585876  0.002560  0.582227  0.588776   
31     6        l-m           0.581854  0.005783  0.569388  0.588355   
32     6        lms           0.583760  0.005445  0.569827  0.587755   
33     6        mel           0.585774  0.002859  0.578571  0.588776   
34     6          s           0.573121  0.037136  0.468367  0.588776   
35     7       flux           0.587245  0.001004  0.585714  0.588776   
36     7        l-m           0.586896  0.000841  0.585291  0.588175   
37     7        lms           0.587475  0.000881  0.586106  0.588957   
38     7        mel           0.586992  0.001217  0.584270  0.588355   
39     7          s           0.586429  0.001408  0.583673  0.588175   
40     8       flux           0.584995  0.003726  0.573034  0.587334   
41     8        l-m           0.586815  0.001771  0.583078  0.589377   
42     8        lms           0.549629  0.141078  0.039712  0.588776   
43     8        mel           0.585686  0.002122  0.579592  0.588776   
44     8          s           0.585222  0.002550  0.581040  0.588776   
45     9       flux           0.580786  0.008789  0.564730  0.587755   
46     9        l-m           0.580732  0.007102  0.568648  0.588175   
47     9        lms           0.579903  0.011237  0.556345  0.589195   
48     9        mel           0.583471  0.005017  0.572895  0.586735   
49     9          s           0.581723  0.008170  0.567623  0.588776   
50    10       flux           0.586343  0.002377  0.580777  0.588776   
51    10        l-m           0.587756  0.000945  0.586137  0.588776   
52    10        lms           0.585348  0.0

### Participant 201

In [13]:
trial_stats_dict["201"]

Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.528952  0.057842  0.445918  0.586510   
1      0        l-m           0.525290  0.092868  0.360572  0.586735   
2      0        lms           0.439848  0.155516  0.235714  0.578571   
3      0        mel           0.424165  0.106904  0.297959  0.559184   
4      0          s           0.475015  0.100987  0.356122  0.585714   
5      1       flux           0.571568  0.014195  0.541544  0.586735   
6      1        l-m           0.573634  0.008165  0.559063  0.584270   
7      1        lms           0.569588  0.015842  0.542390  0.584149   
8      1        mel           0.568938  0.014017  0.532990  0.582653   
9      1          s           0.558986  0.026414  0.497477  0.585714   
10     2       flux           0.577925  0.011421  0.547959  0.587755   
11     2        l-m           0.580958  0.007548  0.563265  0.587755   
12     2        lms           0.576822  0.013501  0.551859  0.588355   
13     2        mel           0.575147  0.010388  0.549540  0.586313   
14     2          s           0.580280  0.006163  0.566769  0.587755   
15     3       flux           0.572058  0.009924  0.556122  0.586735   
16     3        l-m           0.569100  0.019020  0.544343  0.588776   
17     3        lms           0.570045  0.012772  0.552605  0.586313   
18     3        mel           0.575866  0.014056  0.552147  0.587755   
19     3          s           0.576201  0.015484  0.534694  0.587334   
20     4       flux           0.532701  0.068313  0.373469  0.587755   
21     4        l-m           0.563742  0.029895  0.505607  0.587935   
22     4        lms           0.521270  0.103613  0.272727  0.589377   
23     4        mel           0.550084  0.057988  0.393878  0.588595   
24     4          s           0.553318  0.042699  0.459735  0.587156   
25     5       flux           0.466989  0.119508  0.228805  0.580235   
26     5        l-m           0.398179  0.179454  0.147959  0.567347   
27     5        lms           0.432097  0.125422  0.235714  0.580612   
28     5        mel           0.456648  0.110467  0.267620  0.587755   
29     5          s           0.454585  0.147823  0.142857  0.582059   
30     6       flux           0.577821  0.006419  0.564862  0.583673   
31     6        l-m           0.568829  0.016920  0.537755  0.586735   
32     6        lms           0.573445  0.015987  0.539246  0.586735   
33     6        mel           0.576782  0.010998  0.553626  0.585714   
34     6          s           0.564820  0.014452  0.539326  0.581040   
35     7       flux           0.569319  0.022867  0.528571  0.588175   
36     7        l-m           0.563531  0.041798  0.452041  0.587755   
37     7        lms           0.546749  0.061402  0.411645  0.587334   
38     7        mel           0.549516  0.059232  0.419816  0.586735   
39     7          s           0.530555  0.091797  0.327551  0.587755   
40     8       flux           0.585793  0.001560  0.583078  0.588175   
41     8        l-m           0.582527  0.010681  0.553061  0.588776   
42     8        lms           0.583531  0.004317  0.575510  0.587156   
43     8        mel           0.584159  0.006866  0.565306  0.589796   
44     8          s           0.580266  0.011444  0.559184  0.589980   
45     9       flux           0.580716  0.011221  0.558163  0.588355   
46     9        l-m           0.586266  0.002833  0.579162  0.589377   
47     9        lms           0.584736  0.001931  0.581633  0.588355   
48     9        mel           0.583179  0.007248  0.568367  0.588355   
49     9          s           0.584949  0.005083  0.570991  0.588355   
50    10       flux           0.578082  0.009877  0.554536  0.587935   
51    10        l-m           0.579001  0.007711  0.563710  0.587935   
52    10        lms           0.573056  0.0

### Participant 202

In [14]:
trial_stats_dict["202"]

Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.574079  0.021569  0.526531  0.588776   
1      0        l-m           0.584167  0.003841  0.576531  0.587755   
2      0        lms           0.581327  0.013478  0.546476  0.588355   
3      0        mel           0.577857  0.017654  0.532653  0.588355   
4      0          s           0.581938  0.005718  0.570991  0.588776   
5      1       flux           0.569740  0.030290  0.500000  0.588776   
6      1        l-m           0.561076  0.042138  0.462245  0.588175   
7      1        lms           0.572082  0.040542  0.459267  0.589377   
8      1        mel           0.564405  0.072475  0.358163  0.589377   
9      1          s           0.567305  0.049895  0.426531  0.588776   
10     2       flux           0.552419  0.060832  0.388379  0.588776   
11     2        l-m           0.539263  0.084725  0.360572  0.588355   
12     2        lms           0.515374  0.098042  0.291539  0.588063   
13     2        mel           0.565834  0.037941  0.462717  0.588776   
14     2          s           0.544097  0.089436  0.315306  0.587755   
15     3       flux           0.577539  0.010310  0.557594  0.588063   
16     3        l-m           0.584448  0.007050  0.565306  0.589377   
17     3        lms           0.578499  0.010183  0.560204  0.586735   
18     3        mel           0.582424  0.006492  0.570408  0.588175   
19     3          s           0.583954  0.003551  0.576531  0.587334   
20     4       flux           0.575885  0.012976  0.552041  0.587755   
21     4        l-m           0.584023  0.002681  0.579592  0.587334   
22     4        lms           0.573459  0.021675  0.519388  0.587334   
23     4        mel           0.575025  0.021642  0.520940  0.587576   
24     4          s           0.572331  0.020987  0.530071  0.587755   
25     5       flux           0.544629  0.060543  0.440816  0.588355   
26     5        l-m           0.543258  0.066352  0.396534  0.588175   
27     5        lms           0.557543  0.036629  0.498980  0.587755   
28     5        mel           0.561801  0.031813  0.493878  0.587334   
29     5          s           0.550944  0.041118  0.464760  0.587755   
30     6       flux           0.557924  0.053433  0.421859  0.586735   
31     6        l-m           0.541175  0.085256  0.338776  0.588355   
32     6        lms           0.567341  0.034731  0.484200  0.587755   
33     6        mel           0.581326  0.007894  0.567347  0.587755   
34     6          s           0.561019  0.032536  0.505618  0.586735   
35     7       flux           0.584288  0.008875  0.559633  0.588776   
36     7        l-m           0.568325  0.034748  0.498471  0.588776   
37     7        lms           0.585982  0.002872  0.580235  0.588776   
38     7        mel           0.586710  0.001911  0.582227  0.588776   
39     7          s           0.586259  0.002817  0.580184  0.589377   
40     8       flux           0.578191  0.011359  0.553061  0.588776   
41     8        l-m           0.583038  0.010663  0.554082  0.588957   
42     8        lms           0.575072  0.017274  0.536735  0.587156   
43     8        mel           0.579259  0.016443  0.538776  0.588776   
44     8          s           0.579131  0.013385  0.545918  0.587935   
45     9       flux           0.586852  0.001037  0.584694  0.588355   
46     9        l-m           0.586914  0.001622  0.583248  0.588355   
47     9        lms           0.587737  0.001056  0.585533  0.588776   
48     9        mel           0.587298  0.001418  0.585291  0.589377   
49     9          s           0.584694  0.008285  0.561224  0.588355   
50    10       flux           0.451427  0.187667  0.043922  0.587755   
51    10        l-m           0.409758  0.198469  0.041794  0.587755   
52    10        lms           0.450808  0.1

### Participant 204

In [15]:
trial_stats_dict["204"]

Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.334070  0.123977  0.159346  0.514872   
1      0        l-m           0.345828  0.120167  0.119388  0.516327   
2      0        lms           0.390501  0.086170  0.253320  0.536188   
3      0        mel           0.349284  0.098104  0.215526  0.503575   
4      0          s           0.344764  0.153032  0.164286  0.559184   
5      1       flux           0.501211  0.066533  0.365949  0.577120   
6      1        l-m           0.476377  0.094352  0.254341  0.567485   
7      1        lms           0.448238  0.137507  0.167347  0.578571   
8      1        mel           0.439074  0.148675  0.208970  0.584270   
9      1          s           0.476320  0.113060  0.250765  0.580020   
10     2       flux           0.485670  0.119136  0.210419  0.578141   
11     2        l-m           0.539411  0.030699  0.495403  0.584694   
12     2        lms           0.499715  0.122291  0.265036  0.583248   
13     2        mel           0.452706  0.110385  0.291837  0.577551   
14     2          s           0.518128  0.067268  0.347959  0.578141   
15     3       flux           0.474850  0.100071  0.288776  0.567347   
16     3        l-m           0.487144  0.043537  0.398573  0.553061   
17     3        lms           0.471448  0.058396  0.348980  0.539796   
18     3        mel           0.506324  0.044296  0.399387  0.571429   
19     3          s           0.489948  0.046085  0.397344  0.548980   
20     4       flux           0.444949  0.086736  0.322778  0.531154   
21     4        l-m           0.408419  0.114031  0.241062  0.548980   
22     4        lms           0.434573  0.147269  0.072449  0.543412   
23     4        mel           0.430944  0.128431  0.146939  0.562245   
24     4          s           0.471809  0.052043  0.375894  0.553517   
25     5       flux           0.404586  0.080117  0.295918  0.560204   
26     5        l-m           0.411503  0.110601  0.167347  0.525510   
27     5        lms           0.354512  0.124758  0.130746  0.505618   
28     5        mel           0.425038  0.041302  0.374490  0.486239   
29     5          s           0.425215  0.107838  0.156282  0.516327   
30     6       flux           0.384883  0.133843  0.185728  0.546939   
31     6        l-m           0.414482  0.101252  0.277835  0.573469   
32     6        lms           0.334238  0.137656  0.070336  0.467824   
33     6        mel           0.436678  0.125645  0.167347  0.566905   
34     6          s           0.363189  0.125731  0.096840  0.511224   
35     7       flux           0.302935  0.109807  0.139869  0.537634   
36     7        l-m           0.277413  0.128025  0.038462  0.487232   
37     7        lms           0.311355  0.161823  0.019608  0.555102   
38     7        mel           0.306101  0.147888  0.064987  0.530612   
39     7          s           0.298390  0.142944  0.093872  0.517347   
40     8       flux           0.359497  0.129894  0.177370  0.556428   
41     8        l-m           0.413041  0.084727  0.276531  0.533129   
42     8        lms           0.414034  0.096308  0.242363  0.560327   
43     8        mel           0.400225  0.071945  0.275510  0.496425   
44     8          s           0.458129  0.092268  0.283963  0.564155   
45     9       flux           0.398910  0.142163  0.068437  0.556751   
46     9        l-m           0.381250  0.127287  0.153218  0.526047   
47     9        lms           0.369758  0.149161  0.039755  0.505102   
48     9        mel           0.400517  0.091577  0.213048  0.510725   
49     9          s           0.363000  0.159594  0.036810  0.546939   
50    10       flux           0.537702  0.023638  0.481595  0.567788   
51    10        l-m           0.530267  0.014651  0.506122  0.554082   
52    10        lms           0.532857  0.0

### Participant 205

In [16]:
trial_stats_dict["205"]

Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.291062  0.093891  0.147959  0.395301   
1      0        l-m           0.238022  0.147908  0.046987  0.473469   
2      0        lms           0.231269  0.137532  0.013265  0.436290   
3      0        mel           0.137548  0.114029  0.000000  0.315306   
4      0          s           0.180009  0.118950  0.051020  0.443878   
5      1       flux           0.327086  0.148841  0.164969  0.528571   
6      1        l-m           0.305395  0.132800  0.095821  0.501020   
7      1        lms           0.295430  0.212377  0.000000  0.537755   
8      1        mel           0.270514  0.144867  0.000000  0.439796   
9      1          s           0.312951  0.143791  0.098879  0.548980   
10     2       flux           0.283780  0.207547  0.072375  0.552605   
11     2        l-m           0.270164  0.158526  0.000000  0.482125   
12     2        lms           0.249472  0.184588  0.025536  0.542390   
13     2        mel           0.209806  0.142400  0.058104  0.508163   
14     2          s           0.244097  0.219177  0.010215  0.552605   
15     3       flux           0.285422  0.104082  0.157303  0.502935   
16     3        l-m           0.252136  0.133750  0.101020  0.530612   
17     3        lms           0.334565  0.106393  0.211224  0.518367   
18     3        mel           0.277041  0.159755  0.065306  0.539796   
19     3          s           0.321759  0.123604  0.112245  0.525026   
20     4       flux           0.135704  0.073342  0.036772  0.267347   
21     4        l-m           0.167665  0.158229  0.000000  0.428571   
22     4        lms           0.111500  0.103388  0.000000  0.347959   
23     4        mel           0.160020  0.087891  0.036772  0.335714   
24     4          s           0.136252  0.097183  0.000000  0.272727   
25     5       flux           0.193231  0.150390  0.000000  0.401631   
26     5        l-m           0.229457  0.145345  0.000000  0.435714   
27     5        lms           0.247800  0.133369  0.057201  0.403061   
28     5        mel           0.267909  0.143541  0.082737  0.503061   
29     5          s           0.282882  0.122092  0.109295  0.491318   
30     6       flux           0.346761  0.142909  0.114402  0.512219   
31     6        l-m           0.365870  0.149017  0.141982  0.540816   
32     6        lms           0.370142  0.134795  0.134694  0.552041   
33     6        mel           0.359124  0.131795  0.095918  0.502554   
34     6          s           0.353403  0.123809  0.139653  0.555102   
35     7       flux           0.434127  0.074010  0.348315  0.511747   
36     7        l-m           0.457844  0.034364  0.425510  0.509704   
37     7        lms           0.450803  0.089052  0.352400  0.562072   
38     7        mel           0.331091  0.120379  0.175510  0.469867   
39     7          s           0.387884  0.064706  0.298264  0.474490   
40     8       flux           0.469047  0.053210  0.366972  0.545455   
41     8        l-m           0.405038  0.078608  0.273469  0.511224   
42     8        lms           0.441343  0.093154  0.261224  0.548924   
43     8        mel           0.352323  0.112970  0.201020  0.542857   
44     8          s           0.391748  0.108190  0.229826  0.546381   
45     9       flux           0.299121  0.146356  0.000000  0.482125   
46     9        l-m           0.309950  0.108496  0.150000  0.463812   
47     9        lms           0.355048  0.138963  0.082569  0.508665   
48     9        mel           0.386967  0.088213  0.260204  0.553517   
49     9          s           0.294520  0.122009  0.108053  0.436735   
50    10       flux           0.317908  0.186331  0.043922  0.547959   
51    10        l-m           0.259871  0.161192  0.041794  0.548980   
52    10        lms           0.329607  0.2

### Participant 206

In [17]:
trial_stats_dict["206"]

Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.543894  0.048693  0.412844  0.580612   
1      0        l-m           0.552026  0.032475  0.488776  0.584694   
2      0        lms           0.558053  0.015065  0.534219  0.578141   
3      0        mel           0.544670  0.023688  0.494898  0.569388   
4      0          s           0.544362  0.053585  0.418367  0.580184   
5      1       flux           0.382675  0.077408  0.335072  0.586735   
6      1        l-m           0.361508  0.080733  0.311360  0.586735   
7      1        lms           0.392411  0.102642  0.314694  0.588776   
8      1        mel           0.397347  0.129491  0.250965  0.581633   
9      1          s           0.392564  0.098404  0.333333  0.579592   
10     2       flux           0.568264  0.010603  0.550459  0.582031   
11     2        l-m           0.569416  0.009754  0.558733  0.588355   
12     2        lms           0.566970  0.026139  0.496425  0.586137   
13     2        mel           0.546648  0.086046  0.302349  0.581633   
14     2          s           0.545565  0.060323  0.394898  0.586137   
15     3       flux           0.535801  0.043775  0.477495  0.585117   
16     3        l-m           0.511073  0.090153  0.303061  0.586313   
17     3        lms           0.493921  0.122035  0.199796  0.583673   
18     3        mel           0.519245  0.061213  0.397959  0.581040   
19     3          s           0.496504  0.104297  0.255363  0.580612   
20     4       flux           0.546239  0.054100  0.404494  0.579824   
21     4        l-m           0.565433  0.017834  0.533197  0.582653   
22     4        lms           0.564172  0.013449  0.542857  0.583673   
23     4        mel           0.562805  0.023165  0.522936  0.586735   
24     4          s           0.568119  0.018509  0.535714  0.587755   
25     5       flux           0.468773  0.101345  0.338446  0.577551   
26     5        l-m           0.482524  0.076199  0.341488  0.564286   
27     5        lms           0.512070  0.048626  0.438776  0.559755   
28     5        mel           0.520522  0.044223  0.427988  0.581205   
29     5          s           0.512091  0.055208  0.417347  0.569969   
30     6       flux           0.500497  0.092215  0.274210  0.580020   
31     6        l-m           0.498832  0.057227  0.367347  0.549080   
32     6        lms           0.494664  0.072390  0.320408  0.554082   
33     6        mel           0.452065  0.077718  0.351682  0.567347   
34     6          s           0.495761  0.056498  0.400409  0.558733   
35     7       flux           0.464069  0.146433  0.183953  0.587755   
36     7        l-m           0.511646  0.122419  0.186925  0.586313   
37     7        lms           0.442451  0.188309  0.109295  0.585714   
38     7        mel           0.403971  0.184386  0.106122  0.579592   
39     7          s           0.449728  0.161691  0.213265  0.585714   
40     8       flux           0.570622  0.010672  0.549439  0.585117   
41     8        l-m           0.563225  0.016288  0.533673  0.587755   
42     8        lms           0.560345  0.020284  0.521429  0.580020   
43     8        mel           0.575294  0.008143  0.556575  0.583673   
44     8          s           0.563091  0.036294  0.464286  0.588776   
45     9       flux           0.576627  0.019499  0.532176  0.588776   
46     9        l-m           0.582713  0.006540  0.571429  0.587755   
47     9        lms           0.585226  0.003940  0.575510  0.588776   
48     9        mel           0.582673  0.008158  0.567347  0.589377   
49     9          s           0.579855  0.019145  0.525510  0.588175   
50    10       flux           0.555521  0.039572  0.477017  0.587755   
51    10        l-m           0.540082  0.056727  0.438776  0.588175   
52    10        lms           0.568301  0.0

### Participant 207

In [18]:
trial_stats_dict["207"]

Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.482174  0.110785  0.228571  0.566327   
1      0        l-m           0.536172  0.043428  0.458163  0.573469   
2      0        lms           0.493154  0.122063  0.176351  0.572885   
3      0        mel           0.491493  0.152344  0.083673  0.575510   
4      0          s           0.525567  0.076139  0.325510  0.573469   
5      1       flux           0.537935  0.012934  0.513790  0.552497   
6      1        l-m           0.545678  0.015885  0.524490  0.570408   
7      1        lms           0.537735  0.021399  0.494898  0.565749   
8      1        mel           0.526084  0.057584  0.372069  0.560652   
9      1          s           0.544495  0.013387  0.515832  0.560204   
10     2       flux           0.327102  0.156249  0.000000  0.531633   
11     2        l-m           0.447240  0.077563  0.321757  0.530133   
12     2        lms           0.412993  0.134618  0.086823  0.518897   
13     2        mel           0.421120  0.154193  0.000000  0.555669   
14     2          s           0.386396  0.123183  0.132653  0.518367   
15     3       flux           0.382867  0.126587  0.170421  0.540816   
16     3        l-m           0.358568  0.089758  0.212029  0.495413   
17     3        lms           0.402589  0.075457  0.295199  0.535168   
18     3        mel           0.379404  0.114707  0.199183  0.541837   
19     3          s           0.415291  0.086841  0.255363  0.514286   
20     4       flux           0.128713  0.100649  0.000000  0.348980   
21     4        l-m           0.190070  0.094147  0.047959  0.365953   
22     4        lms           0.214201  0.136551  0.008163  0.416327   
23     4        mel           0.176214  0.106287  0.000000  0.365953   
24     4          s           0.141180  0.131078  0.012232  0.416327   
25     5       flux           0.294774  0.104662  0.130612  0.428571   
26     5        l-m           0.305298  0.116655  0.105209  0.447395   
27     5        lms           0.252561  0.149745  0.000000  0.412245   
28     5        mel           0.308491  0.120226  0.114286  0.492857   
29     5          s           0.245908  0.137818  0.037717  0.483673   
30     6       flux           0.421920  0.140507  0.086735  0.547959   
31     6        l-m           0.426201  0.120977  0.181448  0.551020   
32     6        lms           0.375468  0.142842  0.165475  0.533197   
33     6        mel           0.381313  0.113485  0.129724  0.507143   
34     6          s           0.443452  0.105909  0.186735  0.556122   
35     7       flux           0.398756  0.139900  0.052147  0.561224   
36     7        l-m           0.447762  0.079413  0.335714  0.563265   
37     7        lms           0.419853  0.104543  0.231672  0.557712   
38     7        mel           0.399310  0.158767  0.002843  0.561798   
39     7          s           0.385982  0.182283  0.070408  0.559184   
40     8       flux           0.407734  0.139502  0.113381  0.539326   
41     8        l-m           0.372681  0.194395  0.085714  0.551583   
42     8        lms           0.422041  0.155958  0.032653  0.547959   
43     8        mel           0.379503  0.148554  0.088866  0.551020   
44     8          s           0.385334  0.176648  0.057143  0.534694   
45     9       flux           0.413319  0.176487  0.039755  0.554536   
46     9        l-m           0.511324  0.013241  0.497452  0.533673   
47     9        lms           0.411509  0.140247  0.184694  0.541837   
48     9        mel           0.471722  0.056738  0.355465  0.535714   
49     9          s           0.378539  0.198366  0.035751  0.555556   
50    10       flux           0.356618  0.097643  0.166496  0.477017   
51    10        l-m           0.390069  0.099473  0.235714  0.512768   
52    10        lms           0.286740  0.1

### Participant 209

In [19]:
trial_stats_dict["209"]

Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.221087  0.103753  0.009184  0.376915   
1      0        l-m           0.201019  0.147234  0.000000  0.458163   
2      0        lms           0.181820  0.143837  0.000000  0.383044   
3      0        mel           0.236619  0.147836  0.017365  0.469388   
4      0          s           0.195369  0.153918  0.000000  0.418534   
5      1       flux           0.238458  0.160376  0.000000  0.576531   
6      1        l-m           0.223056  0.164333  0.042857  0.486735   
7      1        lms           0.260162  0.139297  0.000000  0.530071   
8      1        mel           0.288901  0.174226  0.000000  0.588355   
9      1          s           0.211563  0.169855  0.000000  0.587334   
10     2       flux           0.343203  0.144818  0.090816  0.566536   
11     2        l-m           0.379838  0.162783  0.181818  0.576098   
12     2        lms           0.312289  0.172741  0.044898  0.557712   
13     2        mel           0.347851  0.194889  0.040816  0.564286   
14     2          s           0.317398  0.164323  0.026558  0.541369   
15     3       flux           0.204926  0.177041  0.012270  0.451943   
16     3        l-m           0.129258  0.122371  0.000000  0.400000   
17     3        lms           0.192075  0.178857  0.000000  0.440816   
18     3        mel           0.224586  0.201990  0.000000  0.548057   
19     3          s           0.175141  0.135311  0.029592  0.461224   
20     4       flux           0.268422  0.153478  0.000000  0.517347   
21     4        l-m           0.238856  0.119246  0.033673  0.420570   
22     4        lms           0.285490  0.197142  0.015306  0.580184   
23     4        mel           0.269864  0.174859  0.000000  0.534694   
24     4          s           0.290824  0.179859  0.000000  0.523469   
25     5       flux           0.505234  0.055056  0.387755  0.552838   
26     5        l-m           0.522656  0.028254  0.456033  0.553061   
27     5        lms           0.518806  0.039222  0.441267  0.567347   
28     5        mel           0.513835  0.039748  0.424490  0.552605   
29     5          s           0.486471  0.052168  0.386108  0.533197   
30     6       flux           0.071367  0.098485  0.000000  0.281920   
31     6        l-m           0.037669  0.059717  0.000000  0.195918   
32     6        lms           0.043616  0.062657  0.000000  0.167689   
33     6        mel           0.105877  0.157335  0.000000  0.409786   
34     6          s           0.063750  0.097885  0.000000  0.250765   
35     7       flux           0.033302  0.083719  0.000000  0.267620   
36     7        l-m           0.009795  0.018774  0.000000  0.048980   
37     7        lms           0.025102  0.074496  0.000000  0.236735   
38     7        mel           0.007037  0.011822  0.000000  0.031633   
39     7          s           0.047245  0.063642  0.000000  0.182840   
40     8       flux           0.358678  0.177887  0.165306  0.585117   
41     8        l-m           0.431452  0.125310  0.216327  0.572012   
42     8        lms           0.411542  0.137604  0.190622  0.572449   
43     8        mel           0.417058  0.094478  0.251020  0.580612   
44     8          s           0.395971  0.181673  0.080612  0.567347   
45     9       flux           0.323920  0.122727  0.116327  0.475996   
46     9        l-m           0.308668  0.128453  0.080612  0.476483   
47     9        lms           0.396765  0.100634  0.258317  0.542304   
48     9        mel           0.369113  0.164309  0.009193  0.559184   
49     9          s           0.415897  0.065525  0.318043  0.538776   
50    10       flux           0.388007  0.142512  0.161224  0.534694   
51    10        l-m           0.381885  0.114727  0.195097  0.487755   
52    10        lms           0.341591  0.1

### Participant 210

In [20]:
trial_stats_dict["210"]

Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.493834  0.044927  0.408163  0.555669   
1      0        l-m           0.473511  0.074685  0.342186  0.553626   
2      0        lms           0.494385  0.067865  0.372829  0.580645   
3      0        mel           0.476460  0.063800  0.368744  0.558733   
4      0          s           0.499944  0.044910  0.429592  0.576531   
5      1       flux           0.324656  0.138124  0.080695  0.481409   
6      1        l-m           0.304096  0.154293  0.020387  0.569827   
7      1        lms           0.343425  0.112757  0.158325  0.547959   
8      1        mel           0.310111  0.103183  0.158163  0.458716   
9      1          s           0.387319  0.073490  0.275229  0.517347   
10     2       flux           0.458377  0.077985  0.284694  0.547959   
11     2        l-m           0.464124  0.089853  0.292857  0.551583   
12     2        lms           0.477832  0.059154  0.356513  0.541284   
13     2        mel           0.438267  0.090487  0.295918  0.527068   
14     2          s           0.467041  0.061043  0.365953  0.547959   
15     3       flux           0.344674  0.174820  0.104082  0.547959   
16     3        l-m           0.326208  0.148085  0.127681  0.550459   
17     3        lms           0.299630  0.200409  0.030333  0.562245   
18     3        mel           0.326624  0.194589  0.047910  0.543878   
19     3          s           0.317217  0.154057  0.087755  0.510204   
20     4       flux           0.288219  0.187299  0.051935  0.563841   
21     4        l-m           0.363789  0.113842  0.146939  0.479103   
22     4        lms           0.260341  0.111943  0.126402  0.478571   
23     4        mel           0.348548  0.121313  0.138776  0.478039   
24     4          s           0.257944  0.114732  0.127291  0.483673   
25     5       flux           0.296582  0.147087  0.138776  0.551020   
26     5        l-m           0.409795  0.143515  0.192865  0.582227   
27     5        lms           0.251195  0.170530  0.026558  0.523955   
28     5        mel           0.288744  0.142783  0.022449  0.487755   
29     5          s           0.346353  0.158055  0.030822  0.565306   
30     6       flux           0.312075  0.155050  0.101020  0.522449   
31     6        l-m           0.396454  0.118132  0.159346  0.531633   
32     6        lms           0.393776  0.156888  0.107143  0.525510   
33     6        mel           0.308566  0.129116  0.138776  0.498471   
34     6          s           0.351877  0.140806  0.131498  0.512742   
35     7       flux           0.227854  0.091827  0.113265  0.383187   
36     7        l-m           0.277022  0.082650  0.133810  0.407747   
37     7        lms           0.326425  0.127981  0.103166  0.496939   
38     7        mel           0.283622  0.153928  0.090909  0.540816   
39     7          s           0.256323  0.123834  0.101020  0.480612   
40     8       flux           0.291753  0.099484  0.174668  0.468846   
41     8        l-m           0.286146  0.113684  0.128571  0.504082   
42     8        lms           0.268318  0.141665  0.000000  0.450000   
43     8        mel           0.229173  0.116162  0.042857  0.380224   
44     8          s           0.244960  0.139499  0.128571  0.575510   
45     9       flux           0.296155  0.150342  0.093878  0.489796   
46     9        l-m           0.437840  0.067330  0.313265  0.560776   
47     9        lms           0.284695  0.122708  0.141837  0.443878   
48     9        mel           0.318047  0.130939  0.164286  0.536188   
49     9          s           0.366419  0.082937  0.220408  0.521472   
50    10       flux           0.239098  0.173110  0.017329  0.525994   
51    10        l-m           0.336685  0.122662  0.161060  0.513265   
52    10        lms           0.179664  0.1

### Participant 211

In [21]:
trial_stats_dict["211"]

Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.540020  0.029194  0.496939  0.588776   
1      0        l-m           0.561674  0.014540  0.541369  0.580612   
2      0        lms           0.557013  0.026686  0.494129  0.580612   
3      0        mel           0.534857  0.048196  0.420838  0.586735   
4      0          s           0.518464  0.035170  0.459653  0.555102   
5      1       flux           0.410159  0.179134  0.176711  0.584555   
6      1        l-m           0.482322  0.110446  0.250000  0.575077   
7      1        lms           0.459043  0.121048  0.257900  0.577982   
8      1        mel           0.451669  0.156856  0.205912  0.586313   
9      1          s           0.495152  0.092551  0.307143  0.581633   
10     2       flux           0.549606  0.034468  0.489796  0.581205   
11     2        l-m           0.565388  0.012084  0.543412  0.577120   
12     2        lms           0.545629  0.026065  0.497446  0.581040   
13     2        mel           0.562547  0.020098  0.523469  0.588175   
14     2          s           0.544841  0.029502  0.489275  0.581040   
15     3       flux           0.564803  0.028449  0.491892  0.588776   
16     3        l-m           0.565448  0.016288  0.536735  0.585291   
17     3        lms           0.549711  0.027904  0.499490  0.578571   
18     3        mel           0.559190  0.017169  0.533673  0.577551   
19     3          s           0.560230  0.030194  0.492355  0.583078   
20     4       flux           0.522337  0.058990  0.375894  0.570552   
21     4        l-m           0.474391  0.177081  0.071356  0.576531   
22     4        lms           0.545223  0.034334  0.470408  0.582059   
23     4        mel           0.464966  0.161466  0.085714  0.580184   
24     4          s           0.521728  0.088049  0.285714  0.583248   
25     5       flux           0.506609  0.054911  0.427115  0.573469   
26     5        l-m           0.486809  0.065170  0.357873  0.566327   
27     5        lms           0.528250  0.048664  0.467824  0.581372   
28     5        mel           0.499807  0.070044  0.364286  0.573034   
29     5          s           0.492355  0.075630  0.358818  0.573904   
30     6       flux           0.467302  0.063560  0.341837  0.539796   
31     6        l-m           0.424286  0.110009  0.308163  0.563265   
32     6        lms           0.480147  0.081890  0.300306  0.572012   
33     6        mel           0.448525  0.085400  0.355759  0.584694   
34     6          s           0.501902  0.076599  0.360204  0.577551   
35     7       flux           0.491861  0.094434  0.278571  0.582227   
36     7        l-m           0.459988  0.133116  0.238776  0.585117   
37     7        lms           0.504238  0.050383  0.436159  0.579162   
38     7        mel           0.529855  0.031993  0.490816  0.580612   
39     7          s           0.474463  0.099814  0.293878  0.586313   
40     8       flux           0.553857  0.025115  0.501018  0.574619   
41     8        l-m           0.555954  0.046433  0.434694  0.586313   
42     8        lms           0.562871  0.024235  0.507645  0.583248   
43     8        mel           0.549785  0.038722  0.467890  0.583078   
44     8          s           0.553728  0.026312  0.494393  0.578571   
45     9       flux           0.581011  0.013715  0.543878  0.588355   
46     9        l-m           0.575515  0.016280  0.539715  0.587755   
47     9        lms           0.578486  0.009082  0.566769  0.587755   
48     9        mel           0.574336  0.020957  0.528090  0.589377   
49     9          s           0.579609  0.007217  0.567347  0.588355   
50    10       flux           0.567471  0.019470  0.519388  0.587334   
51    10        l-m           0.577114  0.007071  0.569388  0.588355   
52    10        lms           0.571491  0.0

### Participant 212

In [22]:
trial_stats_dict["212"]

Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.465358  0.170293  0.139796  0.588776   
1      0        l-m           0.497236  0.135597  0.226531  0.588776   
2      0        lms           0.432850  0.145903  0.152905  0.585714   
3      0        mel           0.423180  0.166166  0.098059  0.587755   
4      0          s           0.456728  0.161136  0.144898  0.587755   
5      1       flux           0.401213  0.162715  0.095918  0.588776   
6      1        l-m           0.376368  0.192010  0.035714  0.588175   
7      1        lms           0.397758  0.202895  0.043922  0.588776   
8      1        mel           0.372205  0.207443  0.033639  0.586735   
9      1          s           0.370447  0.245114  0.000000  0.587755   
10     2       flux           0.498617  0.185058  0.000000  0.587755   
11     2        l-m           0.558489  0.066332  0.374490  0.588776   
12     2        lms           0.514350  0.175057  0.020492  0.587755   
13     2        mel           0.516570  0.182272  0.000000  0.589195   
14     2          s           0.422828  0.241270  0.000000  0.588355   
15     3       flux           0.586907  0.001621  0.583248  0.588355   
16     3        l-m           0.573512  0.023555  0.520408  0.588776   
17     3        lms           0.575117  0.021387  0.532587  0.588355   
18     3        mel           0.586345  0.002419  0.580020  0.588776   
19     3          s           0.584922  0.002968  0.580777  0.588355   
20     4       flux           0.561490  0.045695  0.467347  0.588063   
21     4        l-m           0.583180  0.006571  0.568367  0.589377   
22     4        lms           0.585744  0.003078  0.579162  0.588776   
23     4        mel           0.583852  0.004045  0.576531  0.588175   
24     4          s           0.578835  0.020337  0.521429  0.587755   
25     5       flux           0.578989  0.011237  0.555102  0.588776   
26     5        l-m           0.578249  0.007316  0.563710  0.587156   
27     5        lms           0.582317  0.006057  0.568948  0.588355   
28     5        mel           0.583752  0.005522  0.572449  0.589377   
29     5          s           0.569792  0.019325  0.528571  0.588776   
30     6       flux           0.435284  0.131652  0.140059  0.587755   
31     6        l-m           0.334064  0.183277  0.101020  0.550562   
32     6        lms           0.463160  0.104663  0.291539  0.587334   
33     6        mel           0.446425  0.132501  0.198980  0.585714   
34     6          s           0.491317  0.091162  0.310521  0.586735   
35     7       flux           0.551814  0.102189  0.261224  0.588776   
36     7        l-m           0.529222  0.086067  0.364286  0.588776   
37     7        lms           0.584325  0.002567  0.579592  0.586735   
38     7        mel           0.548237  0.067779  0.392638  0.588355   
39     7          s           0.578373  0.020189  0.522449  0.588776   
40     8       flux           0.581835  0.011328  0.555669  0.588776   
41     8        l-m           0.554312  0.055938  0.414286  0.587755   
42     8        lms           0.584629  0.003444  0.578571  0.587755   
43     8        mel           0.583573  0.007379  0.563710  0.588175   
44     8          s           0.580504  0.020279  0.522983  0.588355   
45     9       flux           0.560729  0.047740  0.431633  0.586313   
46     9        l-m           0.492137  0.137168  0.256881  0.586137   
47     9        lms           0.555530  0.063552  0.389173  0.586912   
48     9        mel           0.561401  0.052741  0.414286  0.588355   
49     9          s           0.572372  0.020684  0.518367  0.585714   
50    10       flux           0.560638  0.042952  0.450000  0.587084   
51    10        l-m           0.407519  0.240504  0.018443  0.588957   
52    10        lms           0.533763  0.1

### Participant 213

In [23]:
trial_stats_dict["213"]

Block Trial type Stim eye - Size Mm                                \
                      count/size ratio                                 
                                  mean       std       min       max   
0      0       flux           0.394958  0.164286  0.000000  0.586735   
1      0        l-m           0.427859  0.216280  0.000000  0.581205   
2      0        lms           0.294717  0.228111  0.000000  0.544343   
3      0        mel           0.308478  0.267565  0.000000  0.553626   
4      0          s           0.358891  0.207142  0.039837  0.587755   
5      1       flux           0.573166  0.014302  0.548519  0.588776   
6      1        l-m           0.521177  0.066643  0.375510  0.584270   
7      1        lms           0.556398  0.039936  0.457696  0.587755   
8      1        mel           0.564490  0.020926  0.523955  0.587755   
9      1          s           0.546438  0.086355  0.307143  0.588355   
10     2       flux           0.538914  0.088767  0.358529  0.589377   
11     2        l-m           0.549537  0.048573  0.444898  0.587334   
12     2        lms           0.547768  0.102487  0.259184  0.588776   
13     2        mel           0.479022  0.181698  0.113265  0.587755   
14     2          s           0.531725  0.104765  0.263534  0.588776   
15     3       flux           0.540664  0.082332  0.315306  0.588465   
16     3        l-m           0.477357  0.074696  0.352041  0.581633   
17     3        lms           0.512539  0.072811  0.378571  0.587755   
18     3        mel           0.531045  0.099031  0.267347  0.588355   
19     3          s           0.490395  0.129096  0.197757  0.588175   
20     4       flux           0.538781  0.077039  0.338776  0.584694   
21     4        l-m           0.531208  0.091173  0.297242  0.584694   
22     4        lms           0.520400  0.090544  0.323829  0.588776   
23     4        mel           0.542236  0.050875  0.429009  0.583673   
24     4          s           0.557197  0.045789  0.447395  0.587334   
25     5       flux           0.422249  0.137966  0.166496  0.587755   
26     5        l-m           0.415266  0.154994  0.169388  0.580448   
27     5        lms           0.427414  0.123908  0.231397  0.586735   
28     5        mel           0.407032  0.170721  0.036735  0.573469   
29     5          s           0.374133  0.206123  0.040858  0.566769   
30     6       flux           0.446868  0.147713  0.114286  0.576098   
31     6        l-m           0.306524  0.172731  0.045918  0.584694   
32     6        lms           0.398185  0.138736  0.185714  0.542304   
33     6        mel           0.361063  0.172855  0.085802  0.587334   
34     6          s           0.294056  0.201102  0.020429  0.588776   
35     7       flux           0.397976  0.159342  0.074414  0.588776   
36     7        l-m           0.307473  0.172396  0.059184  0.529592   
37     7        lms           0.328277  0.192345  0.029592  0.566769   
38     7        mel           0.249641  0.157123  0.072523  0.523469   
39     7          s           0.418433  0.091150  0.290816  0.546939   
40     8       flux           0.305391  0.238390  0.014300  0.588776   
41     8        l-m           0.240460  0.201419  0.000000  0.574490   
42     8        lms           0.281936  0.229727  0.009184  0.586735   
43     8        mel           0.250937  0.212454  0.012257  0.587334   
44     8          s           0.322946  0.246647  0.016327  0.588776   
45     9       flux           0.377430  0.153590  0.122449  0.574490   
46     9        l-m           0.387325  0.194240  0.110317  0.587334   
47     9        lms           0.494385  0.122888  0.165306  0.588776   
48     9        mel           0.486947  0.141356  0.193054  0.585714   
49     9          s           0.472377  0.127200  0.224490  0.586735   
50    10       flux           0.511241  0.097277  0.291837  0.588063   
51    10        l-m           0.456704  0.075164  0.337079  0.586735   
52    10        lms           0.541410  0.1